In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
train_users = pd.read_csv("../data/raw/train_users_2.csv")
display(train_users.head())
print(train_users.shape)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


(213451, 16)


In [3]:
test_users = pd.read_csv("../data/raw/test_users.csv")
display(test_users.head())
print(test_users.shape)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,20140701000051,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,20140701000148,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
3,6c6puo6ix0,2014-07-01,20140701000215,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
4,czqhjk3yfe,2014-07-01,20140701000305,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari


(62096, 15)


In [4]:
def df_feature_analysis(df):
    
    for feature in df.columns.values.tolist():
        print(feature)
        print(df[feature].value_counts())
        print('Contains NaN: ', df[feature].isnull().values.any())
        print('\n')

In [5]:
def clean_up(df):
    
    # remove date_first_booking feature
    df = df.drop(['date_first_booking'], axis = 1)
    
    # split date account created feature into seperate year, month and day columns
    df[["create_year", "create_month", "create_day"]] = df["date_account_created"].str.split("-", expand=True).astype('int64')
    df = df.drop(['date_account_created'], axis = 1)
    
    # locate outlier ages and replace with nan values
    df.loc[df['age'] > 90, 'age'] = np.nan
    df.loc[df['age'] < 16, 'age'] = np.nan
    
    # replace nan age values with mean ages
    df.loc[df['age'].isnull(), 'age'] = int(df['age'].mean())
    
    df.loc[df['first_affiliate_tracked'].isnull(), 'first_affiliate_tracked'] = 'untracked'
    return df

def setup_encode(df):
    
    df_cat = df.select_dtypes(include=[object])
    enc = OneHotEncoder(handle_unknown='ignore',sparse=False)
    enc_fit = enc.fit(df_cat)
    
    return enc_fit

def encode(df, encoding):
    
    df_cat = df.select_dtypes(include=[object])
    df_num = df.select_dtypes(exclude=[object])
    encoded_data = encoding.transform(df_cat)
    encoded_df = pd.DataFrame(encoded_data)
    
    df = pd.concat([df_num, encoded_df], axis=1)
    
    return df

def setup_scale(df):
    
    df_scale = df[['timestamp_first_active', 'age', 'signup_flow', 'create_year', 'create_month', 'create_day']]
    scaler = StandardScaler()
    scale_fit = scaler.fit(df_scale)
    
    return scale_fit

def scale(df, scaling):
    
    df_scale = df[['timestamp_first_active', 'age', 'signup_flow', 'create_year', 'create_month', 'create_day']]
    df_no_scale = df.drop(['timestamp_first_active', 'age', 'signup_flow', 'create_year', 'create_month', 'create_day'], axis=1)
    scaled_data = scaling.transform(df_scale)
    scaled_df = pd.DataFrame(scaled_data, columns = list(df_scale.columns))
    
    df = pd.concat([scaled_df, df_no_scale], axis=1)
    
    return df

In [6]:
df_targets = train_users[['country_destination']] 
df_train = clean_up(train_users).drop(['id','country_destination'], axis = 1)
df_test = clean_up(test_users).drop(['id'], axis = 1)

In [7]:
display(df_train)

,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,create_year,create_month,create_day
0,20090319043255,-unknown-,36.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2010,6,28
1,20090523174809,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,2011,5,25
2,20090609231247,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,2010,9,28
3,20091031060129,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,2011,12,5
4,20091208061105,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2010,9,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213446,20140630235636,MALE,32.0,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Safari,2014,6,30
213447,20140630235719,-unknown-,36.0,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,2014,6,30
213448,20140630235754,-unknown-,32.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,2014,6,30
213449,20140630235822,-unknown-,36.0,basic,25,en,other,other,tracked-other,iOS,iPhone,Mobile Safari,2014,6,30


In [8]:
output = pd.concat([df_train, df_targets], axis=1)
output.to_csv('cat_clean_train.csv', index=False)

df_feature_analysis(df_train)

df_feature_analysis(df_test)

In [9]:
encoded_train = encode(df_train, setup_encode(df_train))
encoded_test = encode(df_test, setup_encode(df_train))

In [10]:
display(encoded_train)

,timestamp_first_active,age,signup_flow,create_year,create_month,create_day,0,1,2,3,...,120,121,122,123,124,125,126,127,128,129
0,20090319043255,36.0,0,2010,6,28,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20090523174809,38.0,0,2011,5,25,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20090609231247,56.0,3,2010,9,28,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20091031060129,42.0,0,2011,12,5,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20091208061105,41.0,0,2010,9,14,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213446,20140630235636,32.0,0,2014,6,30,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
213447,20140630235719,36.0,0,2014,6,30,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
213448,20140630235754,32.0,0,2014,6,30,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
213449,20140630235822,36.0,25,2014,6,30,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
display(encoded_test)

,timestamp_first_active,age,signup_flow,create_year,create_month,create_day,0,1,2,3,...,120,121,122,123,124,125,126,127,128,129
0,20140701000006,35.0,0,2014,7,1,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20140701000051,34.0,0,2014,7,1,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20140701000148,34.0,0,2014,7,1,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20140701000215,34.0,0,2014,7,1,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20140701000305,34.0,0,2014,7,1,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62091,20140930235232,31.0,0,2014,9,30,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62092,20140930235306,34.0,23,2014,9,30,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62093,20140930235408,34.0,0,2014,9,30,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62094,20140930235430,34.0,0,2014,9,30,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
scaled_train = scale(encoded_train, setup_scale(encoded_train))
display(scaled_train)

,timestamp_first_active,age,signup_flow,create_year,create_month,create_day,0,1,2,3,...,120,121,122,123,124,125,126,127,128,129
0,-4.380020,-0.030801,-0.427798,-3.222044,-0.006939,1.387946,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-4.357961,0.199273,-0.427798,-2.156499,-0.315897,1.044700,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-4.348661,2.269939,-0.035009,-3.222044,0.919936,1.387946,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-4.303076,0.659421,-0.427798,-2.156499,1.846811,-1.243607,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-4.283949,0.544384,-0.427798,-3.222044,0.919936,-0.213869,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213446,1.056855,-0.490950,-0.427798,1.040136,-0.006939,1.616777,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
213447,1.056855,-0.030801,-0.427798,1.040136,-0.006939,1.616777,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
213448,1.056855,-0.490950,-0.427798,1.040136,-0.006939,1.616777,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
213449,1.056855,-0.030801,2.845444,1.040136,-0.006939,1.616777,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
scaled_test = scale(encoded_test, setup_scale(encoded_train))
display(scaled_test)

,timestamp_first_active,age,signup_flow,create_year,create_month,create_day,0,1,2,3,...,120,121,122,123,124,125,126,127,128,129
0,1.064503,-0.145838,-0.427798,1.040136,0.302019,-1.701268,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.064503,-0.260875,-0.427798,1.040136,0.302019,-1.701268,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.064503,-0.260875,-0.427798,1.040136,0.302019,-1.701268,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.064503,-0.260875,-0.427798,1.040136,0.302019,-1.701268,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.064503,-0.260875,-0.427798,1.040136,0.302019,-1.701268,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62091,1.089275,-0.605987,-0.427798,1.040136,0.919936,1.616777,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62092,1.089275,-0.260875,2.583584,1.040136,0.919936,1.616777,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62093,1.089275,-0.260875,-0.427798,1.040136,0.919936,1.616777,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62094,1.089275,-0.260875,-0.427798,1.040136,0.919936,1.616777,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
output = pd.concat([scaled_train, df_targets], axis=1)
output.to_csv('encoded_scaled_train.csv', index=False)